In [24]:
import pandas as pd
from scipy.io import savemat
import matlab.engine

def create_segment(shift):
    n_initial_240s = 22 + shift
    n_150s = 19
    n_final_240s = 1344 - n_initial_240s - n_150s
    
    segment = [240] * n_initial_240s + [150] * n_150s + [240] * n_final_240s
    return segment

def generate_sequence(length):
    seq = []
    value = 0
    for _ in range(length):
        seq.append(value)
        value += 1/96
        if round(value, 2) >= 14:
            value = 0
    return seq

def save_to_mat(data):
    df = pd.DataFrame(data, columns=['Value'])
    df['Sequence'] = generate_sequence(len(df))

    # Validate the data shape
    if df.shape != (1344, 2):
        raise ValueError(f"Unexpected data shape: {df.shape}. Expected (1344, 2).")
    
    # Create combined 2D list
    combined_data = df[['Sequence', 'Value']].values.tolist()
    
    # Save this combined list as a single variable in the .mat file
    data_dict_tank3 = {'KLa3_Setpoints_14Day': combined_data}
    data_dict_tank4 = {'KLa4_Setpoints_14Day': combined_data}
    data_dict_tank5 = {'KLa5_Setpoints_14Day': combined_data}
    
    savemat('ASM1-2d-3_MATLAB/ASM1 2d 3 in BSM1/ASM1/KLa5_Setpoints_14Day.mat', data_dict_tank5)
    savemat('ASM1-2d-3_MATLAB/ASM1 2d 3 in BSM1/ASM1/KLa4_Setpoints_14Day.mat', data_dict_tank4)
    savemat('ASM1-2d-3_MATLAB/ASM1 2d 3 in BSM1/ASM1/KLa3_Setpoints_14Day.mat', data_dict_tank3)

def run_matlab_model(iteration):
    eng = matlab.engine.start_matlab()

    # Save the 'iteration' value as a MATLAB datafile
    iteration_data = {"iteration": iteration}
    savemat('ASM1-2d-3_MATLAB/ASM1 2d 3 in BSM1/ASM1/iteration.mat', iteration_data)

    eng.cd('/Users/aya/github/WWDR/ASM1-2d-3_MATLAB/ASM1 2d 3 in BSM1/ASM1', nargout=0)
    eng.run('run_TSaeration_DRbsm1.m', nargout=0)
    eng.quit()

# Get the number of iterations
n_segments = int(input("Enter the number of iterations: "))

shift = 0
for i in range(n_segments):
    data = create_segment(shift)
    save_to_mat(data)
    run_matlab_model(i + 1)  # Pass the current iteration count, starting from 1
    shift += 4


Current iteration value is: 1
 
Running BSM1 to steady state! Solver = ode15s and Simulink model = benchmarkss
**************************************************************************
 
Steady state achieved. Initializing all state variables to steady state values.
 
Simulating BSM1 with dynamic influent (i) in closed loop (Tempmodel = 1)! Solver = ode45 and Simulink model = benchmark
*****************************************************************************************************************
 
Start time for simulation (hour:min:sec) = 17  54  26
Simulating BSM1 with dynamic influent (ii) in closed loop (Tempmodel = 1)! Solver = ode45 and Simulink model = benchmark
*****************************************************************************************************************
 
Start time for simulation (hour:min:sec) = 17  54  54
Dynamic closed loop BSM1 simulation finished!
End time for simulation (hour:min:sec) = 17  55  20
 
Current iteration value is: 2
 
Running BSM1 to s